<h1>Using the API</h1>
In this notebook, we will see how to use the Spotipy to navigate the Spotify API.

---

<h2>Prerequisites</h2>
<ol>
    <li><a href = 'https://nbviewer.org/github/JonYarber/music_modeling/blob/main/1.%20Setting%20Up%20the%20API%20Connection.ipynb'>Setting up the API Connection</a></li>
</ol>

---

<h3>Connect to Spotify</h3>
Let's create a function that will conveniently get us connected to the Spotify API using the method outlined in the <a href = 'https://nbviewer.org/github/JonYarber/music_modeling/blob/main/1.%20Setting%20Up%20the%20API%20Connection.ipynb'>Setting up the API Connection</a>.

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

def connect_to_spotify():
    sp_client_id = input("Enter your Spotify Client ID: ")
    sp_client_secret = input("Enter your Spotify Secret Token: ")
    
    credentials = SpotifyClientCredentials(client_id = sp_client_id,
                                           client_secret = sp_client_secret)
    
    sp = spotipy.Spotify(client_credentials_manager = credentials)
    
    print("Connected to Spotify!")

    return sp

In [4]:
sp = connect_to_spotify()

Enter your Spotify Client ID:  ed9307841d3542df8819aec9a4f0ec84
Enter your Spotify Secret Token:  8208717955574be6a24163ed59675094


Connected to Spotify!


<h3>Using the Search Function</h3>
It all starts with a search!<br>
Spotify identifies tracks and artists with their own unique resource identifier or URI. The URI is key to retrieving desired artist or track information. We find these URIs by utilizing the <b>search</b> method of our connection object, <b>sp</b>. Refer to the <a href = 'https://spotipy.readthedocs.io/en/2.24.0/#spotipy.client.Spotify.search'>Spotipy documentation</a> for complete information regarding the <b>search</b> feature.<br>
<br>
For this project, we will be performing a <i>track</i> search by setting the <b>type</b> parameter to <b>'track'</b>. Here is an example of the syntax:

<br>
There are three reasons for utilizing search type <b>'track'</b> on this project:<br>
<ol>
    <li>We need are modeling songs, so we need a track URI.</li>
    <li>We can obtain both the track and artist URI from a track search, if needed.</li>
    <li>We can add the name of the artist to the search term.</li>
</ol>
The search will always return the specified number of results (the <b>limit</b> parameter, default 10) regardless of how good a match the result is. To ensure the first result is the correct result, that third item is important.

<h3>Performing a Search</h3>
Assume we want the URI for the song <i>Without You</i> by Mariah Carey. Without specifying the artist name and just executing a search for the track 'Without You', we get the following results:<br>
<i>(Ignore syntax for the time being. We will cover this next.)</i>

In [53]:
for item in sp.search('Without You', type = 'track', limit = 15)['tracks']['items']:
    print(f'Artist: {item['artists'][0]['name']}')
    print(f'Track: {item['name']}')
    print('-' * 30)

Artist: The Kid LAROI
Track: WITHOUT YOU
------------------------------
Artist: Rosario Dawson
Track: Without You
------------------------------
Artist: David Guetta
Track: Without You (feat. Usher)
------------------------------
Artist: George Lamond
Track: Without You
------------------------------
Artist: Avicii
Track: Without You (feat. Sandro Cavazza)
------------------------------
Artist: DJ Max Star
Track: WITHOUT YOU (Freestyle)
------------------------------
Artist: Strawberry Guy
Track: Without You
------------------------------
Artist: Billi Royce
Track: Without You
------------------------------
Artist: Chris Stapleton
Track: Without Your Love
------------------------------
Artist: Grant
Track: Without You
------------------------------
Artist: 3 Doors Down
Track: Here Without You
------------------------------
Artist: Inhavit
Track: Without You
------------------------------
Artist: Matt Hansen
Track: without you with me
------------------------------
Artist: Luvbird
Track

<br>I had to set the <b>limit</b> to 15 just to get the result we wanted! If this was the only way to retrieve that track URI, we would have to add some frustrating layers to our processing. However, we can avoid that and ensure that we get the correct result by informing both the artist and track name in the search term as follows:

In [51]:
# The spacing is important!
search_term = f'artist:Mariah Carey track:Without You'

# Let's just see what the first result returns
search_result = sp.search(search_term, type = 'track')['tracks']['items'][0]

print(f'Artist: {search_result['artists'][0]['name']}')
print(f'Track: {search_result['name']}')

Artist: Mariah Carey
Track: Without You


<h3>Parsing the JSON</h3>
Now that we know how to use <b>search</b>, let's see how to get what we need from the result.<br>
As I'm sure you've already gathered, the results of the search are in JSON format. We will be examining how to retrieve <i>very specific</i> items from the result. If you are unfamiliar with JSON dictionaries I highly recommend giving <a href='https://en.wikipedia.org/wiki/JSON'>this</a> a quick read. I will also be displaying the JSON results in a pandas dataframe format for readability.<br>
<br>
Before proceeding let's store the search results of 'Without You' into a variable.

In [64]:
search_results = sp.search('Without You', limit = 15)

<h4>Layer 1: 'tracks'</h4>
With 15 results specificied, we would expect the length of our dictionary to be 15. However, it is not:

In [65]:
len(search_results)

1

This is because everything the very first term the dictionary, 'tracks', stores information regarding the query itself. Here's a better view:

In [73]:
import pandas as pd
pd.DataFrame(search_results)

,tracks
href,https://api.spotify.com/v1/search?query=Withou...
items,"[{'album': {'album_type': 'album', 'artists': ..."
limit,15
next,https://api.spotify.com/v1/search?query=Withou...
offset,0
previous,None
total,899


To get what we need, we need to go down another layer.

<h4>Layer 2: 'items'</h4>
From the above dataframe we see that the information we need appears in 'items':

In [74]:
len(search_results['tracks']['items'])

15

That's what we want to see. This means that any time we perform a search, we can immediately go down two layers to 'items'

In [92]:
# Redo search result, except parse JSON down to 'items'
search_results = sp.search('Without You', limit = 15)['tracks']['items']

# View search results (first 5)
pd.DataFrame(search_results).head()

,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track_number,type,uri
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",2,161384,True,{'isrc': 'USSM12006586'},{'spotify': 'https://open.spotify.com/track/1K...,https://api.spotify.com/v1/tracks/1KMkcUvF7m3S...,1KMkcUvF7m3SDChDOa7i5L,False,WITHOUT YOU,74,https://p.scdn.co/mp3-preview/0cacbf2acaef9fc5...,7,track,spotify:track:1KMkcUvF7m3SDChDOa7i5L
1,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,255033,False,{'isrc': 'USQY51478307'},{'spotify': 'https://open.spotify.com/track/4e...,https://api.spotify.com/v1/tracks/4e0SLDU1gDDy...,4e0SLDU1gDDyJSscUmA8hm,False,Without You (feat. Nichole),35,https://p.scdn.co/mp3-preview/9ed747c6dddb84b2...,16,track,spotify:track:4e0SLDU1gDDyJSscUmA8hm
2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, PR, US]",1,208133,False,{'isrc': 'GB28K1100030'},{'spotify': 'https://open.spotify.com/track/1t...,https://api.spotify.com/v1/tracks/1tAutlhI9Fwn...,1tAutlhI9FwnKn7fVgmd5P,False,Without You (feat. Usher),64,https://p.scdn.co/mp3-preview/d4c235ddc1029890...,4,track,spotify:track:1tAutlhI9FwnKn7fVgmd5P
3,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,256746,False,{'isrc': 'USBYQ0500020'},{'spotify': 'https://open.spotify.com/track/2b...,https://api.spotify.com/v1/tracks/2btI9j6ov2Ig...,2btI9j6ov2IgC5tAacIGvQ,False,Without You,38,https://p.scdn.co/mp3-preview/b2886c9c6d30b8d8...,20,track,spotify:track:2btI9j6ov2IgC5tAacIGvQ
4,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,228080,False,{'isrc': 'TCADO1884573'},{'spotify': 'https://open.spotify.com/track/2h...,https://api.spotify.com/v1/tracks/2hIrHUgeI2cK...,2hIrHUgeI2cKYwM18MkGPK,False,Without You,63,https://p.scdn.co/mp3-preview/e3b1578cb8e6d9da...,1,track,spotify:track:2hIrHUgeI2cKYwM18MkGPK


This is where we find our track URI (<b>uri</b>). Knowing how to search for a specific track and artist, how to limit our search results, and how to get down to this layer, we can now easily retrieve this:

In [108]:
search_term = f'artist:Mariah Carey track:Without You'

# Even though we're only specifying 1 result, we still have to specify that item ([0])
specific_search_result = sp.search(search_term, type = 'track', limit = 1)['tracks']['items'][0]

print(f'Track URI: {specific_search_result['uri']}')

Track URI: spotify:track:0pkIJFV6mviH9dmBYsFwTM


<h4>Layer 3: 'artists'</h4>
Even though we feel good about using the song and artist name in the search term, we are still going to want to be sure we got the right artist. For this, we need to go down one more layer to 'artists'

In [103]:
pd.DataFrame(specific_search_result['artists'])

,external_urls,href,id,name,type,uri
0,{'spotify': 'https://open.spotify.com/artist/4...,https://api.spotify.com/v1/artists/4iHNK0tOyZP...,4iHNK0tOyZPYnBU7nGAgpQ,Mariah Carey,artist,spotify:artist:4iHNK0tOyZPYnBU7nGAgpQ


<br>There's what we need in 'name':

In [106]:
print(f'Artist Name: {specific_search_result['artists'][0]['name']}')

Artist Name: Mariah Carey


<h4>Put It All Together</h4>
Let's do one last search in which we put it all together to get the track name, track URI, and artist name.

In [122]:
# Create search term
search_term = f'artist:Beatles track:Hello Goodbye'

# Store result of search
search_result = sp.search(search_term, type = 'track', limit = 1)['tracks']['items'][0]

# Store track name as returned by Spotify
track_title = search_result['name']

# Store track URI
track_uri = search_result['uri']

# Store artist name 
track_artist = search_result['artists'][0]['name']


print(f'Artist:\t\t{track_artist}')
print(f'Track:\t\t{track_title}')
print(f'Track URI:\t{track_uri}')

Artist:		The Beatles
Track:		Hello, Goodbye - Remastered 2009
Track URI:	spotify:track:0vZ97gHhemKm6c64hTfJNA
